In [1]:
! pip install sentence_transformers

     |████████████████████████████████| 85 kB 2.4 MB/s 
     |████████████████████████████████| 2.8 MB 25.8 MB/s 
     |████████████████████████████████| 1.2 MB 40.3 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 636 kB 47.9 MB/s 
     |████████████████████████████████| 3.3 MB 49.9 MB/s 
     |████████████████████████████████| 895 kB 38.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=b6d15370fae2be6d770ffc3f5f96de743c854f41cb1d85d308d977fe93268f0a
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
! pip install beir

     |████████████████████████████████| 49 kB 2.4 MB/s 
     |████████████████████████████████| 4.4 MB 6.9 MB/s 
     |████████████████████████████████| 363 kB 45.4 MB/s 
     |████████████████████████████████| 8.4 MB 23.5 MB/s 
  Created wheel for pytrec-eval: filename=pytrec_eval-0.5-cp37-cp37m-linux_x86_64.whl size=266281 sha256=0419bd143c7e8674a318e0eb1867e410ff7216c88b8fa5504af9d88e5179000a
  Stored in directory: /root/.cache/pip/wheels/42/96/77/0829b8b2606f90f61ba10a51277629d2b615604e122ee932f4
Successfully built pytrec-eval


In [16]:
! pip install faiss-gpu

     |████████████████████████████████| 89.7 MB 19 kB/s 


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [4]:
import os
os.chdir('/content/gdrive/My Drive/Colab Notebooks')

In [ ]:
! ls

1  3  general	huggingface  Sagemaker	      wiki_movie_plots_deduped.csv
2  4  Haystack	MLOps	     Untitled0.ipynb


In [5]:
from beir import util, LoggingHandler
from beir.datasets.data_loader import GenericDataLoader
from beir.generation import QueryGenerator as QGen
from beir.generation.models import QGenModel
from beir.retrieval.train import TrainRetriever
from sentence_transformers import SentenceTransformer, losses, models

import pathlib, os
import logging

import torch

/usr/local/lib/python3.7/dist-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

In [6]:

data_path = "test"

#### Provide the data_path where nfcorpus has been downloaded and unzipped

# loads the 'corpus.jsonl' file
# the code takes both title and text, we dont have title, so just leave them as empty string
# texts = [(self.gen_prefix + doc["title"] + " " + doc["text"]) for doc in corpus]
corpus = GenericDataLoader(data_path).load_corpus()

  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
##############################
#### 1. Query-Generation  ####
##############################

#### question-generation model loading 
model_path = "BeIR/query-gen-msmarco-t5-base-v1"
generator = QGen(model=QGenModel(model_path))

#### Query-Generation using Nucleus Sampling (top_k=25, top_p=0.95) ####
#### https://huggingface.co/blog/how-to-generate
#### Prefix is required to seperate out synthetic queries and qrels from original
prefix = "gen"

#### Generating 3 questions per passage. 
#### Reminder the higher value might produce lots of duplicates
ques_per_passage = 3

#### Generate queries per passage from docs in corpus and save them in original corpus
#### check your datasets folder to find the generated questions, you will find below:
#### 1. datasets/scifact/gen-3-queries.jsonl
#### 2. datasets/scifact/gen-3-qrels/train.tsv

batch_size = 64


2021-09-21 07:50:52 - Use pytorch device: cuda


In [ ]:
# will generate 'gen-queries.jsonl' file, together with a train.tsv which contains the mappings

generator.generate(corpus, output_dir=data_path, ques_per_passage=ques_per_passage, prefix=prefix, batch_size=batch_size)

2021-09-21 07:51:07 - Starting to Generate 3 Questions Per Passage using top-p (nucleus) sampling...
2021-09-21 07:51:07 - Params: top_p = 0.95
2021-09-21 07:51:07 - Params: top_k = 25
2021-09-21 07:51:07 - Params: max_length = 64
2021-09-21 07:51:07 - Params: ques_per_passage = 3
2021-09-21 07:51:07 - Params: batch size = 64


pas:   0%|          | 0/1 [00:00<?, ?it/s]

2021-09-21 07:51:10 - Saving 36 Generated Queries...
2021-09-21 07:51:10 - Saving Generated Queries to test/gen-queries.jsonl
2021-09-21 07:51:10 - Saving Generated Qrels to test/gen-qrels/train.tsv


In [7]:


corpus_path = os.path.join(data_path, "corpus.jsonl")
query_path = os.path.join(data_path, "gen-queries.jsonl")
qrels_path = os.path.join(data_path, "gen-qrels", "train.tsv")

corpus, queries, qrels = GenericDataLoader(
    corpus_file=corpus_path, 
    query_file=query_path, 
    qrels_file=qrels_path).load_custom()

  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
print(corpus)
print(queries)
print(qrels)

{'MED-10': {'text': 'Recent studies have suggested that statins, an established drug group in the prevention of cardiovascular mortality, could delay or prevent breast cancer recurrence but the effect on disease-specific mortality remains unclear. We evaluated risk of breast cancer death among statin users in a population-based cohort of breast cancer patients. The study cohort included all newly diagnosed breast cancer patients in Finland during 1995–2003 (31,236 cases), identified from the Finnish Cancer Registry. Information on statin use before and after the diagnosis was obtained from a national prescription database. We used the Cox proportional hazards regression method to estimate mortality among statin users with statin use as time-dependent variable. A total of 4,151 participants had used statins. During the median follow-up of 3.25 years after the diagnosis (range 0.08–9.0 years) 6,011 participants died, of which 3,619 (60.2%) was due to breast cancer. After adjustment for a

In [17]:
paragraph_list=[]
with open('combined.tsv', 'w') as f:
  for k, v in qrels.items():
    query = queries[k]
    print(query)
    # paragraph = corpus[v[key]]
    for x in v.keys():
      paragraph = corpus[x]['text']
      score = v[x]
      print(paragraph)
      print(score)
      paragraph_list.append(paragraph)
    f.write("{}\t{}\n".format(query.replace("\t", " ").strip(), paragraph.replace("\t", " ").strip()))

statin safety rates prediagnostic
Recent studies have suggested that statins, an established drug group in the prevention of cardiovascular mortality, could delay or prevent breast cancer recurrence but the effect on disease-specific mortality remains unclear. We evaluated risk of breast cancer death among statin users in a population-based cohort of breast cancer patients. The study cohort included all newly diagnosed breast cancer patients in Finland during 1995–2003 (31,236 cases), identified from the Finnish Cancer Registry. Information on statin use before and after the diagnosis was obtained from a national prescription database. We used the Cox proportional hazards regression method to estimate mortality among statin users with statin use as time-dependent variable. A total of 4,151 participants had used statins. During the median follow-up of 3.25 years after the diagnosis (range 0.08–9.0 years) 6,011 participants died, of which 3,619 (60.2%) was due to breast cancer. After adj

In [10]:
from torch import nn
# Now we create a SentenceTransformer model from scratch
# word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=256)
word_embedding_model = models.Transformer('sentence-transformers/msmarco-distilbert-base-dot-prod-v3', max_seq_length=256)

pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=256, activation_function=nn.Tanh())
# FC layer with tanh activation, on top of the pooling layer
# Hence, embeddings by this model will only have 256 instead of original 768 dimensions.
model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])

In [11]:
from sentence_transformers import SentenceTransformer, InputExample, losses, models, datasets
from torch import nn
import os

train_examples = [] 
with open('combined.tsv') as f:
    for line in f:
        try:
            query, paragraph = line.strip().split('\t', maxsplit=1)
            train_examples.append(InputExample(texts=[query, paragraph]))
        except:
            pass

# For the MultipleNegativesRankingLoss, it is important
# that the batch does not contain duplicate entries, i.e.
# no two equal queries and no two equal paragraphs.
# To ensure this, we use a special data loader
train_dataloader = datasets.NoDuplicatesDataLoader(train_examples, batch_size=8)
# MultipleNegativesRankingLoss requires input pairs (query, relevant_passage)
# and trains the model so that is is suitable for semantic search



# MultipleNegativesRankingLoss is a great loss function if you only have positive pairs, 
# for example, only pairs of similar texts like pairs of paraphrases, pairs of duplicate questions, pairs of (query, response), or pairs of (source_language, target_language).
train_loss = losses.MultipleNegativesRankingLoss(model)

In [12]:
#Tune the model
num_epochs = 3
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=num_epochs, warmup_steps=warmup_steps, show_progress_bar=True)

os.makedirs('test', exist_ok=True)
model.save('test/test-model')

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

In [30]:
# evaluate the embeddings from finetuned model
new_model = SentenceTransformer('test/test-model')
t1 = "Background Cognitive impairment without dementia is associated with increased risk for disability, increased health care costs, and progression to dementia. There are no population-based prevalence estimates of this condition in the United States. Objective To estimate the prevalence of cognitive impairment without dementia in the United States and determine longitudinal cognitive and mortality outcomes. Design Longitudinal study from July 2001 to March 2005. Setting In-home assessment for cognitive impairment. Participants Participants in ADAMS (Aging, Demographics, and Memory Study) who were age 71 years or older drawn from the nationally representative HRS (Health and Retirement Study). Of 1770 selected individuals, 856 completed initial assessment, and of 241 selected individuals, 180 completed 16- to 18-month follow-up assessment. Measurements Assessments, including neuropsychological testing, neurologic examination, and clinical and medical history, were used to assign a diagnosis of normal cognition, cognitive impairment without dementia, or dementia. National prevalence rates were estimated by using a population-weighted sample. Results In 2002, an estimated 5.4 million people (22.2%) in the United States age 71 years or older had cognitive impairment without dementia. Prominent subtypes included prodromal Alzheimer disease (8.2%) and cerebrovascular disease (5.7%). Among participants who completed follow-up assessments, 11.7% with cognitive impairment without dementia progressed to dementia annually, whereas those with subtypes of prodromal Alzheimer disease and stroke progressed at annual rates of 17% to 20%. The annual death rate was 8% among those with cognitive impairment without dementia and almost 15% among those with cognitive impairment due to medical conditions. Limitations Only 56% of the nondeceased target sample completed the initial assessment. Population sampling weights were derived to adjust for at least some of the potential bias due to nonresponse and attrition. Conclusion Cognitive impairment without dementia is more prevalent in the United States than dementia, and its subtypes vary in prevalence and outcomes."
q1 = 'what causes dementia'
t2 = 'Investing in Bitcoin can seem complicated, but it is much easier when you break it down into steps. Buying Bitcoin is getting easier by the day, and the legitimacy of the exchanges and wallets are growing as well.'
q2 = 'how do i invest in bitcoin'

encoding_t1 = torch.from_numpy(new_model.encode([t1]))  # convert numpy intp tensor
encoding_q1 = torch.from_numpy(new_model.encode([q1]))
encoding_t2 = torch.from_numpy(new_model.encode([t2]))
encoding_q2 = torch.from_numpy(new_model.encode([q2]))
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
print(cos(encoding_t1, encoding_q1))
print(cos(encoding_t2, encoding_q2))

print(cos(encoding_t1, encoding_q2))
print(cos(encoding_t2, encoding_q1))


tensor([0.5028])
tensor([0.6949])
tensor([0.1041])
tensor([0.0596])


In [20]:
len(paragraph_list)

36

In [23]:
import faiss
import numpy as np
encoded_data = model.encode(paragraph_list)
encoded_data = np.asarray(encoded_data.astype('float32'))
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
print(encoded_data.shape)
print(np.array(range(0, len(paragraph_list))).shape)
# index.add_with_ids(encoded_data, np.array(range(0, len(paragraph_list))))
# faiss.write_index(index, 'my_test.index')

(36, 256)
(36,)


**We can finetune the generation model**

In [ ]:
#### Training on Generated Queries ####
corpus, gen_queries, gen_qrels = GenericDataLoader(data_path, prefix=prefix).load(split="train")
#### Please Note - not all datasets contain a dev split, comment out the line if such the case
dev_corpus, dev_queries, dev_qrels = GenericDataLoader(data_path).load(split="dev")

#### Provide any HuggingFace model and fine-tune from scratch
model_name = "distilbert-base-uncased" 
word_embedding_model = models.Transformer(model_name, max_seq_length=350)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


#### Or provide already fine-tuned sentence-transformer model
# model = SentenceTransformer("msmarco-distilbert-base-v3")

#### Provide any sentence-transformers model path
model_path = "bert-base-uncased" # or "msmarco-distilbert-base-v3"
retriever = TrainRetriever(model=model, batch_size=64)

#### Prepare training samples
train_samples = retriever.load_train(corpus, gen_queries, gen_qrels)
train_dataloader = retriever.prepare_train(train_samples, shuffle=True)
train_loss = losses.MultipleNegativesRankingLoss(model=retriever.model)

#### Prepare dev evaluator
ir_evaluator = retriever.load_ir_evaluator(dev_corpus, dev_queries, dev_qrels)

#### If no dev set is present evaluate using dummy evaluator
# ir_evaluator = retriever.load_dummy_evaluator()

#### Provide model save path
model_save_path = os.path.join(pathlib.Path(__file__).parent.absolute(), "output", "{}-GenQ-nfcorpus".format(model_path))
os.makedirs(model_save_path, exist_ok=True)

#### Configure Train params
num_epochs = 1
evaluation_steps = 5000
warmup_steps = int(len(train_samples) * num_epochs / retriever.batch_size * 0.1)

retriever.fit(train_objectives=[(train_dataloader, train_loss)], 
                evaluator=ir_evaluator, 
                epochs=num_epochs,
                output_path=model_save_path,
                warmup_steps=warmup_steps,
                evaluation_steps=evaluation_stepmodel_paths,
                use_amp=True)